# Fresh take

## 1. Initalize sites

In [2]:
%load_ext autoreload
import os
import os
import shutil
import dbPipeline
import time
from parseFiles.helperFunctions.loadDict import loadDict

In [3]:
data_dump = r'U:\data-dump\SCL'

In [ ]:
%autoreload 2
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
db = dbPipeline.database(projectPath=projectPath,verbose=True,enableParallel=True)
Sites = r'example_data\projectInventory_Template.yml'
db.projectInventory(newSites=Sites)

template = loadDict(r'example_data\source_Template.yml')


siteID,measurementID = 'SCL','Met'

dpath = 'SCL/2024/20240912'
template[siteID][measurementID]['rootPath'] = os.path.join(data_dump,dpath)
print(template[siteID][measurementID]['rootPath'])
db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])

Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP_metadata.yml
c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP\sourceFiles.yml

does not exist, creating new file using template
line 25:
{'w*ldcard': {'matchPattern': 'w*ldcard', 'rootPath': None, 'fileList': {}, 'parserSettings': {}}}

Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP\sourceFiles.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP_metadata.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP\sourceFiles.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP_metadata.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP\sourceFiles.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP_metadata.yml
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\Sites\_TMP\_TMP\sourceFiles.yml
Saving:

In [ ]:
%autoreload 2

df = dbPipeline.databaseFolder(path=r'C:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\database\SCL\Met',Years=[2024])
df.dataOut.columns

In [ ]:
df.dataOut['NetRad_Avg'].plot()#.to_csv(r'C:\Users\jskeeter\OneDrive - NRCan RNCan\Documents\Presentations\CANCH4\Data\SCL_met.csv')

In [ ]:
from parseFiles.helperFunctions.loadDict import loadDict
# importlib.reload(dbPipeline)

# db = dbPipeline.database(projectPath=projectPath,verbose=False,enableParallel=False)
template = loadDict(r'example_data\source_Template.yml')
# # sourcePath = r'U:\EC_Backup\SCL_2024\20240724'
sourcePath = r'example_data'
print(template)
for siteID in template:
    for measurementID in template[siteID]:
        db.rawFileSearch(siteID,measurementID,template[siteID][measurementID])